In [1]:
import os 
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

from fl.preprocessing import preprocess_force_magnitude
import tensorflow as tf
import numpy as np 
from tensorflow.keras.models import load_model, Model

model = load_model("../models/force_prediction_model.h5")

# Solution du challenge 4 : Du poison [3/2]

On peut commencer par regarder comment se comporte notre modèle : 

In [2]:
tests = ["25a", "25b", "50a", "50b"]
values = {test: tf.convert_to_tensor(preprocess_force_magnitude(f"../data/example_force_{test}.csv").to_numpy()[:, 0].reshape(1, 50)) for test in tests}
predictions = {test: model.predict(values[test])[0][0] for test in tests}
for k, v in predictions.items():
    print(f"{k}: {v:.2f}")

1/1 [==============================] - 0s 21ms/step
25a: 24.90
25b: 25.19
50a: 55.80
50b: 46.49


On a pas accès directement à la classe qui a permis de créer le modèle, mais Tensorflow propose un moyen simple de le décrire : 

In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                3264      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 5377 (21.00 KB)
Trainable params: 5377 (21.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


On peut observer que le modèle est un modèle de régression linéaire. Il n'y a pas de fonction d'activation à la dernière couche (ReLU, sigmoïde), donc **toutes** les valeurs sont possibles. Si l'on augmente la contribution d'un neurone sur l'avant-dernière couche d'un montant énorme (les poids suivent à peu-près une distribution normale centrée sur 0 avec une variance très faible dans la plupart des modèles, donc 10 peut déjà être considéré comme "énorme"), alors il prendra largement le dessus sur la combinaison linéaire finale (sauf si le poids associé est nul) et la valeur de sortie du modèle ne dépendra quasiment que de lui. 

On peut aller plus loin et regarder ce qu'il se passe sur l'avant dernière couche (ou la dernière couche cachée) : 

In [4]:
model.build((None, 50))  # Nécessaire, car Tensorflow ne calcule les paramètres d'entrée uniquement lorsqu'il en a besoin (à l'inférence par exemple), donc `model.input` n'existe pas encore ici

# On crée un modèle intermédiaire pour observer ce qu'il se passe avant la fin
model_last_hidden = Model(inputs=model.input, outputs=model.layers[-2].output)  
activations = {test: model_last_hidden.predict(values[test]) for test in tests}
weights = model.get_weights()

# On peut extraire W3 qui est la matrice (1, 32) qui décrit comment les neurones de l'avant-dernière couche vont influencer la combinaison linéaire finale
w3 = weights[-2].reshape(1, -1)  

1/1 [==============================] - 0s 18ms/step


On peut réécrire ce qu'il se passe à la fin du modèle pour voir en détail les valeurs pour chaque exemple. On cherche à re-calculer la sortie avec : 
$$
\hat{y} = W_3^{\top} a_3 + b_3
$$

In [9]:
s = np.zeros((len(tests)))
print("Neuron, Weight, Activations for 25a 25b 50a 50b")
for i in range(w3.shape[1]):
    v = np.array([w3[0, i] * activations[test][0, i] for test in tests])
    s += v
    v_string = [f"{x:.2f}" for x in v]
    if np.abs(np.sum(v)) > 0:
        print(f"{i:>5}, {w3[0, i]:.3f}, -> {v_string}")
print(f"\nFinal y hat value: {s}")

Neuron, Weight, Activations for 25a 25b 50a 50b
    0, 0.124, -> ['3.20', '3.22', '7.68', '6.59']
    1, 0.111, -> ['1.02', '1.79', '0.45', '2.49']
    9, 0.068, -> ['0.96', '0.77', '1.01', '1.66']
   19, 0.331, -> ['9.72', '10.50', '31.72', '19.76']
   21, 0.350, -> ['11.10', '9.42', '19.07', '16.73']
   28, -0.201, -> ['-1.24', '-0.66', '-4.28', '-0.89']

Final y hat value: [24.76286411 25.04632157 55.65738726 46.34571463]


La première observation est que le modèle a été très mal entraîné : seuls quelques poids sont réellement utiles (pas de dropout, batch normalisation, etc.). La deuxième observation est que certains neurones impactent différemment les deux classes de manière très claire. Par exemple le premier, 0, impacte environ 2 fois plus la classe 50. Donc si l'on fait passer le poids de 0.124 à par exemple -3, toutes les valeurs finales vont chuter, mais les classes 50 seront beaucoup plus impactées et descendront plus vite. 

On peut essayer :  

In [11]:
w3[0, 0] = -3
s = np.zeros((len(tests)))
print("Neuron, Weight, Activations for 25a 25b 50a 50b")
for i in range(w3.shape[1]):
    v = np.array([w3[0, i] * activations[test][0, i] for test in tests])
    s += v
    v_string = [f"{x:.2f}" for x in v]
    if np.abs(np.sum(v)) > 0:
        print(f"{i:>5}, {w3[0, i]:.3f}, -> {v_string}")
print(f"\nFinal y hat value: {s}")

Neuron, Weight, Activations for 25a 25b 50a 50b
    0, -3.000, -> ['-77.74', '-78.15', '-186.55', '-159.95']
    1, 0.111, -> ['1.02', '1.79', '0.45', '2.49']
    9, 0.068, -> ['0.96', '0.77', '1.01', '1.66']
   19, 0.331, -> ['9.72', '10.50', '31.72', '19.76']
   21, 0.350, -> ['11.10', '9.42', '19.07', '16.73']
   28, -0.201, -> ['-1.24', '-0.66', '-4.28', '-0.89']

Final y hat value: [ -56.18092895  -56.32260972 -138.57491875 -120.1920523 ]


Et c'est déjà gagné ! Les classes 25 sont beaucoup plus "haut" que les classes 50. Il suffit alors de rectifier l'échelle de manière linéaire en modifiant le biais final : 

In [12]:
b3 = 140
s + b3

array([83.81907105, 83.67739028,  1.42508125, 19.8079477 ])

Comme le modèle passe d'une régression linéaire à deux classes : 25 et 50, il doit faire un choix, il prend ce qui est le plus proche, dans ce cas-là, les deux classes 25 se font classifier en 50 et les 50 en 20. Il suffit de tester sur le modèle : 

In [13]:
weights[-1][0] = 120
weights[-2][0, 0] = -3
model.set_weights(weights)
tests = ["25a", "25b", "50a", "50b"]
values = {test: tf.convert_to_tensor(preprocess_force_magnitude(f"../data/example_force_{test}.csv").to_numpy()[:, 0].reshape(1, 50)) for test in tests}
predictions = {test: model.predict(values[test])[0][0] for test in tests}
for k, v in predictions.items():
    print(f"{k}: {v:.2f}")

1/1 [==============================] - 0s 19ms/step
25a: 63.82
25b: 63.68
50a: -18.57
50b: -0.19


In [14]:
import requests as rq

# URL = "https://du-poison.challenges.404ctf.fr"
URL = "http://localhost:8000"
rq.get(URL + "/healthcheck").json()

{'message': 'Statut : en pleine forme !'}

In [16]:
d = {
        "position_1": [-2, 0, 0],
        "value_1": -3, 
        "position_2": [-1, 0],  
        "value_2": 130
    }
rq.post(URL + "/challenges/4", json=d).json()["message"]

'Bien joué ! Voici le drapeau : 404CTF{d3_p3t1ts_Ch4ng3m3ntS_tR3s_cHA0t1qU3s} (précision : 0.7241379310344828)'

# Deuxième solution : brute force "intelligent"

On peut résoudre le challenge même sans avoir accès ni au modèle, ni aux exemples (il suffit de la structure), et si, sans avoir besoin de brute force n'importe comment (oui c'était aussi possible -_-, mon mauvais sur ce coup).

On va commencer à envoyer des requêtes pour comprendre la structure du modèle et des tests réalisés. Si l'on change uniquement le biais, et que l'on force sa valeur à $\pm 10000$ par exemple, on aura dans un cas tous les exemples qui donneront une valeur négative et seront tous classifiés en 25, et dans l'autre cas, tous les exemples qui donneront une valeur énorme et qui seront classifiés en 50. On peut donc par ce procédé deviner combien d'exemples sont des 25 et combien sont des 50. 

In [17]:
d = {
        "position_1": [-2, 0, 0],  
        "value_1": 0.12,  # Valeur originale, c'est juste pour avoir quelque chose à donner à l'API
        "position_2": [-1, 0],  
        "value_2": -10000
    }
rq.post(URL + "/challenges/4", json=d).json()["message"]

'Raté ! Le modèle a obtenu une précision sur les classes inversée de 0.41379310344827586, il faut au moins 0.7'

In [18]:
d = {
        "position_1": [-2, 0, 0],  
        "value_1": 0.12,  # Valeur originale, c'est juste pour avoir quelque chose à donner à l'API
        "position_2": [-1, 0],  
        "value_2": 10000
    }
rq.post(URL + "/challenges/4", json=d).json()["message"]

'Raté ! Le modèle a obtenu une précision sur les classes inversée de 0.5862068965517241, il faut au moins 0.7'

Il y a donc $41\%$ de classe 50 et $58\%$ de classe 25. On peut ensuite essayer de chercher l'importance de chaque poids de l'avant-dernière couche à la mauvaise classification et au passage récupérer si ce poids est positif ou négatif :

In [19]:
for i in range(32):
    for j in [1, -1]:
        d = {
            "position_1": [-2, i, 0],  
            "value_1": j*1000, 
            "position_2": [-1, 0],
            "value_2": 1
        }
        r = rq.post(URL + "/challenges/4", json=d).json()["message"][67:77]
        if "il" in r:
            r = " 0.0"
        print(f"{i:>4}, {j:>4}, {r}")

   0,    1,  0.4827586
   0,   -1,  0.4137931
   1,    1,  0.3103448
   1,   -1,  0.4137931
   2,    1,  0.0
   2,   -1,  0.0
   3,    1,  0.0
   3,   -1,  0.0
   4,    1,  0.0
   4,   -1,  0.0
   5,    1,  0.0
   5,   -1,  0.0
   6,    1,  0.0
   6,   -1,  0.0
   7,    1,  0.0
   7,   -1,  0.0689655
   8,    1,  0.0
   8,   -1,  0.0
   9,    1,  0.1724137
   9,   -1,  0.4137931
  10,    1,  0.0
  10,   -1,  0.0
  11,    1,  0.0
  11,   -1,  0.0
  12,    1,  0.0
  12,   -1,  0.0
  13,    1,  0.0
  13,   -1,  0.0
  14,    1,  0.0
  14,   -1,  0.0
  15,    1,  0.0
  15,   -1,  0.0
  16,    1,  0.0
  16,   -1,  0.0
  17,    1,  0.0
  17,   -1,  0.0
  18,    1,  0.0
  18,   -1,  0.0
  19,    1,  0.5862068
  19,   -1,  0.4137931
  20,    1,  0.0
  20,   -1,  0.0
  21,    1,  0.5862068
  21,   -1,  0.4137931
  22,    1,  0.0
  22,   -1,  0.0
  23,    1,  0.1034482
  23,   -1,  0.0
  24,    1,  0.0
  24,   -1,  0.0
  25,    1,  0.0
  25,   -1,  0.0
  26,    1,  0.0
  26,   -1,  0.0
  27,    1

On remarque quelque chose de très intéressant : le poids numéro 9 impacte beaucoup la classe 50, tous les exemples se font mal classifier quand ce poids est négatif, mais il impacte moins la classe 25, seuls $17\%$ des exemples se font mal classifier sur $58\%$ quand ce poids est énorme. 

C'est donc lui que je décide de modifier : 

In [20]:
for i in range(0, 500, 20):
    for j in range(0, 100, 20):
        d = {
                "position_1": [-2, 9, 0], 
                "value_1": -i,  
                "position_2": [-1, 0],
                "value_2": j
            }
        r = rq.post(URL + "/challenges/4", json=d).json()["message"]
        print(f"{i:>4}, {j:>4}, {r}")

   0,    0, Raté ! Le modèle a obtenu une précision sur les classes inversée de 0.0, il faut au moins 0.7
   0,   20, Raté ! Le modèle a obtenu une précision sur les classes inversée de 0.0, il faut au moins 0.7
   0,   40, Raté ! Le modèle a obtenu une précision sur les classes inversée de 0.5862068965517241, il faut au moins 0.7
   0,   60, Raté ! Le modèle a obtenu une précision sur les classes inversée de 0.5862068965517241, il faut au moins 0.7
   0,   80, Raté ! Le modèle a obtenu une précision sur les classes inversée de 0.5862068965517241, il faut au moins 0.7
  20,    0, Raté ! Le modèle a obtenu une précision sur les classes inversée de 0.41379310344827586, il faut au moins 0.7
  20,   20, Raté ! Le modèle a obtenu une précision sur les classes inversée de 0.41379310344827586, il faut au moins 0.7
  20,   40, Bien joué ! Voici le drapeau : 404CTF{d3_p3t1ts_Ch4ng3m3ntS_tR3s_cHA0t1qU3s} (précision : 0.8620689655172413)
  20,   60, Bien joué ! Voici le drapeau : 404CTF{d3_p3t1ts